In [3]:
import specimen
from specimen.classes import medium 

----
# How To: Handle Medium

In this section explains the basic functions for loading, handling and exporting media using the SPECIMEN class Medium. This Jupyter notebook serves as in introduction on how to use this to create and handle your own media when using the SPECIMEN package. For more options and further functionalities for the classes discussed below, please refer to the documentation.

### Creation

A medium (or more precisely the object medium) has three attributes:
- **Name**: Name or better abbreviation of the medium described by a string    
    The name should be short, e.g. M9, and should not contain whitespaces. This will later be used as the identifier in the database.
- **Description**: of the medium, also a string.
- **Compounds**: A dictionary of identifiers (usually the universal BiGG) and Compound objects.    
    For initialisation, a list of Compounds is needed. The identifier are set by the class based on the BiGG ID or formula (see next block for more)
    
Enter a string for the name and description of your medium below:

In [2]:
name = 'test'
description = 'test medium'
compounds = []

Each Compound has the following attributes:

- **Name**: The name of the compound, e.g. Ammonia
- **BiGG ID**: Universal BiGG ID of the compound, e.g. nh4.     
    When using the Compound for a medium, this should be given.
- **formula:** Chemical formula without charge, e.g. NH4 for ammonia.    
    When using the Compound for a medium this should to be given.
- **exchange_flux**: Possible flux of the compound in the media.    
    Optional, if needed, will be set to default 10.0
    
> note: either BiGG ID or formula **MUST** be given to enter the Compound successfully into the Medium

Run the code below with the values for your each of your compounds:

In [3]:
c = medium.Compound()
c.name = 'Ornithine'
c.bigg = 'orn'
c.formula = 'C5H13N2O2'
c.exchange_flux = None
compounds.append(c)

Finally, run the code below to construct the Medium object:

In [6]:
test_medium = medium.Medium(name,description,compounds)

For a better view of your medium, you can convert it into a table:
> note: the the function is used to display whole databases and takes a dictionary of the name and the Medium as input

In [10]:
medium.db_to_table({test_medium.name:test_medium})

,medium,description,compound,bigg_id,formula,exchange_flux
0,test,test medium,Ornithine,orn,C5H13N2O2,None


### Export

**to a file**    
You can save your constructed medium by either saving adding it to an existing database ...

In [11]:
db_path = '/USER/PATH/media.csv'
medium.add_medium_to_db(test_medium, db_path)

... or saving it in a seperate file.
> note: similar to showing the medium as a table, this function takes a dictionary of the name and the Medium as input

In [13]:
out_path = 'your_path.csv'
medium.save_db({test_medium.name:test_medium}, out_path)

**to cobra.Model.medium**  
Additionaly, the medium can be exported into a format that can be used for added the medium to a cobra.Model.

In [ ]:
test_medium.export_to_cobra()

### Import

In addition to creating the Medium by hand, you can also import it from different sources.

#### Option 1: From a database

Import multiple media from a database file that looks the following (dots represent skipped lines):

```
medium;description;compound;bigg_id;formula;exchange_flux
M9;M9_medium (minimal medium) based on protocol from Heilbronner lab;Ca2+;ca2;Ca
.
.
.
LB;Lysogeny broth (from CarveMe);Adenosine;adn;C10H13N5O4;
.
.
.
```

The media are saved as a dictionary with their names as the keys and the loaded Medium objects as the values. Note that the file does not have to contain multiple media.

The in-build database can accessed by passing no file path to `load_media_db()`.

In [4]:
# db_path = '/PATH/TO/USER/DB.csv'
# media_db = medium.load_media_db(db_path)
media_db = medium.load_media_db()
media_db

{'LB': <specimen.classes.medium.Medium at 0x7fa7e9901db0>,
 'LB_o2s': <specimen.classes.medium.Medium at 0x7fa7e1657040>,
 'M9': <specimen.classes.medium.Medium at 0x7fa7e1655420>,
 'M9_o2s': <specimen.classes.medium.Medium at 0x7fa7e1656140>,
 'SNM3': <specimen.classes.medium.Medium at 0x7fa7e9903b50>,
 'SNM3_o2s': <specimen.classes.medium.Medium at 0x7fa7e1645720>,
 'VMH-EUavg': <specimen.classes.medium.Medium at 0x7fa7c01a8610>,
 'VMH-hiFloC': <specimen.classes.medium.Medium at 0x7fa7e1657a90>,
 'VMH-unhealthy': <specimen.classes.medium.Medium at 0x7fa7e1655660>,
 'VMH-vegan': <specimen.classes.medium.Medium at 0x7fa7e16546a0>}

#### Option 2: From a cobra model
If a model has been loaded with *COBRApy*, its medium can be extracted and converted into a Medium object. 

In [ ]:
model = specimen.util.io.read_model_cobra('/PATH/TO/MODEL')
imported_medium = import_medium_from_cobra(model)

### Manipulating

A newly constructed or imported medium (referred to as m in the code examples) can be further manipulated / changes according to the users needs using the following functions:

`m.add_compound(Compound)`    
Add a pre-build Compound to the model.

`m.remove_compound(c)`    
By either using a Compound or a BiGG ID string as input, remove the corresponding compound from the Medium

`m.set_source_of(element, Compound)`    
Searches the medium m for compounds that contain the given element and deletes them as long as they do not serve as
an exclusive source for another element. Afterwards add the new compound to the medium as the new source (compound should contain the element).

`m.make_aerobic()` or `make_anaerobic()`     
Make the medium aerobic or anaerobic by deleting or adding compound(s) with the formula *O2* or the name *o2*.

`m.combine(Medium)` or `+ Medium`     
Combine two mediums into a new one.

In [16]:
# load medium for testing
media_db = medium.load_media_db()
test_medium = media_db['M9']

In [17]:
# generate a compound for testing
c = medium.Compound()
c.name = 'Ornithine'
c.bigg = 'orn'
c.formula = 'C5H13N2O2'

In [18]:
# add a compound
test_medium.add_compound(c)
test_medium.has_compound('orn')
# test_medium.has_compound(c)

True

In [19]:
# remove a compound
test_medium.remove_compound('orn')
# test_medium.remove_compound(c)
test_medium.has_compound('orn')

False

In [20]:
# set source 
print('before: ' + str(test_medium.get_source_of('C')))
test_medium.set_source_of('C', c)
print('after: ' + str(test_medium.get_source_of('C')))

before: ['glc__D', 'cit']
after: ['orn']


In [21]:
# make anaerobic
test_medium.make_anaerobic()
test_medium.is_aerobic()

False

In [22]:
# make aerobic
test_medium.make_aerobic()
test_medium.is_aerobic()

True

In [23]:
# combine two media into a new medium
# --> there are two option on how to do it but both lead to the same result
# in this example, the in-build medium for casamino acids is added to the medium

# option 1:
new_medium = test_medium + medium.CASAMINO_ACIDS

# option 2:
# new_medium = test_medium.combine(medium.CASAMINO_ACIDS)

new_medium.compounds

{'ca2': <missing_name.classes.medium.Compound at 0x7f98ab42af50>,
 'cl': <missing_name.classes.medium.Compound at 0x7f98ab42b010>,
 'cobalt2': <missing_name.classes.medium.Compound at 0x7f98ab42b040>,
 'cu2': <missing_name.classes.medium.Compound at 0x7f98ab42b0a0>,
 'fe2': <missing_name.classes.medium.Compound at 0x7f98ab42b100>,
 'fe3': <missing_name.classes.medium.Compound at 0x7f98ab428ca0>,
 'h2o': <missing_name.classes.medium.Compound at 0x7f98ab42ac50>,
 'h': <missing_name.classes.medium.Compound at 0x7f98ab42ac80>,
 'k': <missing_name.classes.medium.Compound at 0x7f98ab42ab30>,
 'mg2': <missing_name.classes.medium.Compound at 0x7f98ab429f00>,
 'mn2': <missing_name.classes.medium.Compound at 0x7f98ab42a7d0>,
 'mobd': <missing_name.classes.medium.Compound at 0x7f98ab42ad40>,
 'na1': <missing_name.classes.medium.Compound at 0x7f98ab42b220>,
 'nh4': <missing_name.classes.medium.Compound at 0x7f98ab42b280>,
 'ni2': <missing_name.classes.medium.Compound at 0x7f98ab42b2e0>,
 'pi': <mi